In [1]:
import numpy as np
import pandas as pd

In [3]:
xls = pd.ExcelFile('WineKMC.xlsx')

In [6]:
sheet_name = xls.sheet_names
sheet_name

['OfferInformation',
 'Transactions',
 'Pivot',
 'Matrix',
 '4MC',
 '4MC - TopDealsByCluster',
 'Distances',
 '4MC Silhouette',
 '5MC',
 '5MC - TopDealsByCluster',
 '5MC Silhouette',
 '5MedC',
 '5MedC - TopDealsByCluster']

In [9]:
# create dataframe with loop
# exec('{} = pd.DataFrame()'.format(name)) >> got invalid syntax
# https://stackoverflow.com/questions/30635145/create-multiple-dataframes-in-loop

for name in sheet_name:
    globals()["df_" + name] = xls.parse(name)

,Offer #,Campaign,Varietal,Minimum Qty (kg),Discount (%),Origin,Past Peak,Cluster 1,Cluster 2,Cluster 3,...,Turner,Walker,Ward,Watson,White,Williams,Wilson,Wood,Wright,Young
0,1,January,Malbec,72,56,France,FALSE,0.028245,0.012320,0.043462,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN
1,2,January,Pinot Noir,72,17,France,FALSE,0.234237,0.021555,0.107936,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,February,Espumante,144,32,Oregon,TRUE,0.017346,0.022891,0.054139,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,February,Champagne,72,48,France,TRUE,0.015503,0.025705,0.130305,...,1.000000,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,1.000000,NaN
4,5,February,Cabernet Sauvignon,144,44,New Zealand,TRUE,0.022535,0.022249,0.053621,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6,March,Prosecco,144,86,Chile,FALSE,0.027938,0.009625,0.094516,...,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,1.000000
6,7,March,Prosecco,6,40,Australia,TRUE,0.033617,0.540689,0.122585,...,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN
7,8,March,Espumante,6,45,South Africa,FALSE,0.007007,0.430207,0.154885,...,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN
8,9,April,Chardonnay,144,57,Chile,FALSE,0.014457,0.013800,0.141358,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000
9,10,April,Prosecco,72,52,California,FALSE,0.010358,0.041065,0.082892,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN
